In [15]:
!pip install pymupdf
!pip install sentence-transformers
!pip install faiss-cpu
!pip install openai  # or any other LLM provider you prefer


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
import fitz  # PyMuPDF
import re
import os
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import pickle

In [17]:
pdf_path = "/kaggle/input/hsc26-bangla1-pdf/HSC26-Bangla1st-Paper.pdf"

In [18]:
#  Step 1: Load PDF and extract raw text
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [19]:
#  Step 2: Clean the raw text (you can customize this)
def clean_text(text):
    text = re.sub(r'\n+', '\n', text)             # Remove multiple newlines
    text = re.sub(r'[ \t]+', ' ', text)           # Normalize spaces
    text = re.sub(r'\n\s*\n', '\n', text)         # Remove empty lines
    return text.strip()

In [20]:
#  Step 3: Split into chunks (paragraph-based strategy)
def chunk_text(text, max_tokens=100):
    paragraphs = text.split('\n')
    chunks = []
    current_chunk = ""
    for para in paragraphs:
        if len(current_chunk) + len(para.split()) <= max_tokens:
            current_chunk += " " + para
        else:
            chunks.append(current_chunk.strip())
            current_chunk = para
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

In [21]:
#  Step 4: Create embeddings and store in FAISS
def build_vector_store(chunks):
    # Use multilingual embedding model
    model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
    embeddings = model.encode(chunks)
    
    # Convert to numpy float32
    vectors = np.array(embeddings).astype("float32")
    
    # Initialize FAISS index
    index = faiss.IndexFlatL2(vectors.shape[1])
    index.add(vectors)
    
    return index, vectors, chunks

In [22]:
# Main run block
pdf_path = '/kaggle/input/hsc26-bangla1-pdf/HSC26-Bangla1st-Paper.pdf'
raw_text = extract_text_from_pdf(pdf_path)
cleaned_text = clean_text(raw_text)
chunks = chunk_text(cleaned_text, max_tokens=100)
index, vectors, stored_chunks = build_vector_store(chunks)


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

In [23]:
#  Save vector index and chunks for later use
faiss.write_index(index, "bangla_index.faiss")
with open("bangla_chunks.pkl", "wb") as f:
    pickle.dump(stored_chunks, f)

print(f" Extracted {len(chunks)} chunks and stored in FAISS index.")

 Extracted 614 chunks and stored in FAISS index.


In [24]:
import faiss
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer

# Load the same embedding model used before
embed_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Load vector index
index = faiss.read_index("bangla_index.faiss")

# Load chunks
with open("bangla_chunks.pkl", "rb") as f:
    chunks = pickle.load(f)


In [25]:
def search(query, top_k=3):
    # Step 1: Embed the query
    query_embedding = embed_model.encode([query])
    
    # Step 2: Search in FAISS
    D, I = index.search(np.array(query_embedding).astype("float32"), top_k)
    
    # Step 3: Get top matched chunks
    results = [chunks[i] for i in I[0]]
    
    return results

#  Sample Bangla queries
queries = [
    "নজরুল ইসলাম কে ছিলেন?",
    "সাহিত্যের সংজ্ঞা কী?",
    "আধুনিক কবিতা কীভাবে শুরু হয়?",
    "বাংলা সাহিত্যের ধারা",
    "হাজার বছর ধরে উপন্যাসের সারাংশ"
]

#  Run and print results
for q in queries:
    print(f"\n Query: {q}")
    for i, res in enumerate(search(q), 1):
        print(f"{i}. {res.strip()}\n")



 Query: নজরুল ইসলাম কে ছিলেন?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

1. হক কি। ব্যঙ্গাকথি প্রক াগ। আিও আমাকক যদর্খকল মকন হইকব্,  আর্ম অন্নপূণিাি যকাকল গিানকনি  যিাকটা ভাইটি।

2. কলযাণীি সকঙ্গ যদখা হই াকি। হাত যিা়ে কর্ি ার্ি, মাথা যহঁট কর্ি ার্ি; িম্ভুনাথব্াব্ুি হৃদ গজল াকি।

3. মনু-প্রণীত মানুকষি আচিণর্ব্র্ধ সংক্রান্ত গ্রন্থ । প্রিাপর্ত িীকব্ি রষ্ট্া। ব্রহ্মা। ইর্ন র্ব্ক ি যদব্তা।


 Query: সাহিত্যের সংজ্ঞা কী?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

1. মনু-প্রণীত মানুকষি আচিণর্ব্র্ধ সংক্রান্ত গ্রন্থ । প্রিাপর্ত িীকব্ি রষ্ট্া। ব্রহ্মা। ইর্ন র্ব্ক ি যদব্তা।

2. র্নকিি অন্তকিি মকধয শুর্ষ া লই াকিন। অনুপম তাি মামাি চর্িত্র-দব্র্িষ্ট্য প্রসকঙ্গ কথাটি ব্কলকি।

3. কলযাণীি সকঙ্গ যদখা হই াকি। হাত যিা়ে কর্ি ার্ি, মাথা যহঁট কর্ি ার্ি; িম্ভুনাথব্াব্ুি হৃদ গজল াকি।


 Query: আধুনিক কবিতা কীভাবে শুরু হয়?


...

1. 24 ২৪। 'ির়্েমা' িকব্দিঅথিকী? [িা.দিা.’ ১৬] (ক) ির়্েক থাকা  (খ) আ়েষ্ট্তা  (গ) চাকর্চকয  (ঘ) িংধিা

2. গকেি কথক চর্িত্র অনুপকমি আত্মসমাকলাচনা। পর্িমাণ ও গুণ উভ র্দক র্দক ই যে তাি িীব্নটি র্নতান্তই তুে যস

3. ির়্েমা আ়েষ্ট্তা। ি়েত্ব। মঞ্জিী র্কিল েুক্ত কর্চ ডাল। মুকুল একপিন একপ্রি কানপুি কলযাণী যে যদিমাতৃকাি যসব্া আত্মর্নক াগ ককিকি,


 Query: বাংলা সাহিত্যের ধারা


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

1. মনু-প্রণীত মানুকষি আচিণর্ব্র্ধ সংক্রান্ত গ্রন্থ । প্রিাপর্ত িীকব্ি রষ্ট্া। ব্রহ্মা। ইর্ন র্ব্ক ি যদব্তা।

2. নাটক: অচলা তন, র্চিকুমাি সভা, ডাকঘি, মুকুট, মুজক্তি উপা , মুক্তধািা,  িক্তকিব্ী, িািা প্রভৃর্ত।

3. িযাখ্যা: দীপুি চাচা ও ‘অপর্ির্চতা’ গকেি অনুপকমি মামাি যলাভ সীমাহীন। তািা উভক ই যেৌতুককলাভী।


 Query: হাজার বছর ধরে উপন্যাসের সারাংশ


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

1. চলন্ত যিলগার়্েি অর্ব্িাম ধাতব্ ধ্বর্ন যব্াঝাকনা হক কি ধু া গাকনি যে অংি যদাহািিা ব্ািব্াি পর্িকব্িন ককি।

2. র্ব্শ্বর্ব্দযাল কতৃিক র্ড-জলট (১৯৪০)।  জীিনািসান ৭ আগস্ট, ১৯৪১ র্িস্টাব্দ (২২ োব্ণ, ১৩৪৮ ব্ঙ্গাব্দ), যিা়োসঁাককাি ঠাকুিব্ার়্েকত।

3. ওপাকি।চুলকঁাচা, যগঁাকফপাকধর্িকতআিম্ভকর্ি াকি মাত্র।সুপুরুষব্কট।র্ভক়েিমকধযযদর্খকলসককলি আকগতঁািউপকিযচাখ পর়্েব্ািমকতাযচহািা।

